In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

## Load and preprocess train data

In [2]:
# load data
with open('../witt_preprocessing/pickles/dangjin_merged.pkl','rb') as f:
    dangjin_data = pickle.load(f)
with open('../witt_preprocessing/pickles/ulsan_merged.pkl','rb') as f:
    ulsan_data = pickle.load(f)


# preprocess data for modeling

# time as index
dangjin_data.set_index('time', inplace=True)
ulsan_data.set_index('time', inplace=True)

# dangjin - sum target values
dangjin_data['dangjin_sum'] = dangjin_data['dangjin'] + dangjin_data['dangjin_floating'] + dangjin_data['dangjin_warehouse']
dangjin_data.drop(columns=['dangjin','dangjin_floating','dangjin_warehouse'], inplace=True)

# delete where target == 0
dangjin_data = dangjin_data.loc[dangjin_data['dangjin_sum'] != 0]
ulsan_data =  ulsan_data.loc[ulsan_data['ulsan'] != 0]


'''
# delete rows of night (21시부터 6시)
dangjin_data_full = dangjin_data.copy()
ulsan_data_full = ulsan_data.copy()

dangjin_data['hour'] = dangjin_data.index.hour
def f(x):
    if 5 < x < 21:
        return True
    else:
        return False
dangjin_data = dangjin_data.loc[dangjin_data['hour'].apply(lambda x:f(x))]
dangjin_data.drop(columns=['hour'], inplace=True)
'''

"\n# delete rows of night (21시부터 6시)\ndangjin_data_full = dangjin_data.copy()\nulsan_data_full = ulsan_data.copy()\n\ndangjin_data['hour'] = dangjin_data.index.hour\ndef f(x):\n    if 5 < x < 21:\n        return True\n    else:\n        return False\ndangjin_data = dangjin_data.loc[dangjin_data['hour'].apply(lambda x:f(x))]\ndangjin_data.drop(columns=['hour'], inplace=True)\n"

## Load and preprocess test data (public LB)

In [3]:
sample_submission = pd.read_csv('../original_dataset/sample_submission.csv')

# 2월 일자만
sample_submission_feb = sample_submission.loc[sample_submission['time'].str.contains('2021-02')]
sample_submission_feb.head(25)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0,0,0,0
1,2021-02-01 02:00:00,0,0,0,0
2,2021-02-01 03:00:00,0,0,0,0
3,2021-02-01 04:00:00,0,0,0,0
4,2021-02-01 05:00:00,0,0,0,0
5,2021-02-01 06:00:00,0,0,0,0
6,2021-02-01 07:00:00,0,0,0,0
7,2021-02-01 08:00:00,0,0,0,0
8,2021-02-01 09:00:00,0,0,0,0
9,2021-02-01 10:00:00,0,0,0,0


In [4]:
# load
with open('../witt_preprocessing/pickles/dangjin_fcst.pkl','rb') as f:
    dangjin = pickle.load(f)
with open('../witt_preprocessing/pickles/ulsan_fcst.pkl','rb') as f:
    ulsan = pickle.load(f)

# index according to sample submission
start, end  = pd.Timestamp('2021-02-01 01:00:00'), pd.Timestamp('2021-03-01-00:00:00') # end == 2021-02-28 24:00:00

# slice test data
dangjin.set_index('time', inplace=True)
dangjin_feb = dangjin.loc[start:end,:]
print(dangjin_feb.shape)

ulsan.set_index('time', inplace=True)
ulsan_feb = ulsan.loc[start:end,:]
print(ulsan_feb.shape)

(672, 9)
(672, 9)


## Config
### X
- obs보다 fcst가 좋다
- Wind_X와 Wind_Y는 제외하는 것이 좋다
- Temperature는 포함하는 것이 좋다
- Year_sin은 포함하는 것이 좋다
### Hp
- min_samples_leaf = 4

In [5]:
# X and Y cols for train data
x_cols_obs = ['Temperature_obs', 'Humidity_obs','Cloud_obs','Day_cos','Day_sin','Year_cos','Year_sin']
x_cols_fcst = ['Temperature_fcst','Humidity_fcst','Cloud_fcst','Day_cos','Day_sin','Year_cos','Year_sin']

y_dangjin_cols = ['dangjin_sum']
y_ulsan_cols = ['ulsan']

RS = 518

## Predict

In [6]:
def predict(model, train_data, x_cols_train, y_cols_train, test_data, x_cols_test):
    '''
    train_data, test_data: pd.DataFrame
    '''

    # print
    print('='*50)
    print('x_cols_train: ', x_cols_train)
    print('y_cols_train: ', y_cols_train)
    print('x_cols_test: ', x_cols_test)
    print('='*50)

    # input-target split
    x = train_data.loc[:,x_cols_train]
    y = train_data.loc[:,y_cols_train]

    # fit
    model.fit(x,y)

    # predict
    test_x = test_data.loc[:,x_cols_test]
    predict = model.predict(test_x)

    return predict, model

In [8]:
# X columns for test data
print(dangjin_feb.columns)
print(ulsan_feb.columns)

x_cols_feb = ['Temperature', 'Humidity', 'Cloud','Day_cos', 'Day_sin', 'Year_cos', 'Year_sin']

RS=518

Index(['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day_cos',
       'Day_sin', 'Year_cos', 'Year_sin'],
      dtype='object')
Index(['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day_cos',
       'Day_sin', 'Year_cos', 'Year_sin'],
      dtype='object')


In [9]:
# predict dangjin
model = RandomForestRegressor(criterion='mae', verbose=0, n_jobs=-1, random_state=RS, n_estimators=100, min_samples_leaf=6)

dangjin_predict, dangjin_model = predict(model, dangjin_data, x_cols_fcst, y_dangjin_cols, dangjin_feb, x_cols_feb)

x_cols_train:  ['Temperature_fcst', 'Humidity_fcst', 'Cloud_fcst', 'Day_cos', 'Day_sin', 'Year_cos', 'Year_sin']
y_cols_train:  ['dangjin_sum']
x_cols_test:  ['Temperature', 'Humidity', 'Cloud', 'Day_cos', 'Day_sin', 'Year_cos', 'Year_sin']
<ipython-input-6-2eca1d0cd7f3>:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x,y)


In [10]:
# predict ulsan
model = RandomForestRegressor(criterion='mae', verbose=0, n_jobs=-1, random_state=RS, n_estimators=100, min_samples_leaf=6)

ulsan_predict, ulsan_model = predict(model, ulsan_data, x_cols_fcst, y_ulsan_cols, ulsan_feb, x_cols_feb)

x_cols_train:  ['Temperature_fcst', 'Humidity_fcst', 'Cloud_fcst', 'Day_cos', 'Day_sin', 'Year_cos', 'Year_sin']
y_cols_train:  ['ulsan']
x_cols_test:  ['Temperature', 'Humidity', 'Cloud', 'Day_cos', 'Day_sin', 'Year_cos', 'Year_sin']
<ipython-input-6-2eca1d0cd7f3>:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x,y)


In [18]:
print(x_cols_feb)
print(dangjin_model.feature_importances_)
print(ulsan_model.feature_importances_)

['Temperature', 'Humidity', 'Cloud', 'Day_cos', 'Day_sin', 'Year_cos', 'Year_sin']
[0.04819414 0.15213202 0.13695224 0.32140375 0.17133658 0.11958307
 0.0503982 ]
[0.04482682 0.06254054 0.20933256 0.33738162 0.16181768 0.10595228
 0.07814849]


In [14]:
# save model for public LB
with open('./rf_models/dangjin_model.pkl', 'wb') as f:
    pickle.dump(dangjin_model, f)
with open('./rf_models/ulsan_model.pkl', 'wb') as f:
    pickle.dump(ulsan_model, f)

## sample_submission.csv

In [19]:
def to_submission(dangjin_predict, ulsan_predict):
    # sum (doesn't matter)
    predict = dangjin_predict + ulsan_predict
    # add
    sample_submission.iloc[:predict.shape[0],1] = predict

    return sample_submission

In [20]:
def below_320(df):
    # function to apply
    def f(x):
        if x<320:
            return 320
        else:
            return x
    df['dangjin_floating'] = df['dangjin_floating'].apply(lambda x:f(x))
    return df

In [21]:
# below_320
submission_org = to_submission(dangjin_predict, ulsan_predict)
submission_320 = below_320(submission_org)

# to csv
PATH = 'rf_min-leaf-6_train-fcst_itp-lin_no-zero_320.csv'
submission_320.to_csv(PATH, index=False)

# check
pd.read_csv(PATH)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,320.0,0,0,0
1,2021-02-01 02:00:00,320.0,0,0,0
2,2021-02-01 03:00:00,320.0,0,0,0
3,2021-02-01 04:00:00,320.0,0,0,0
4,2021-02-01 05:00:00,320.0,0,0,0
...,...,...,...,...,...
1387,2021-07-08 20:00:00,320.0,0,0,0
1388,2021-07-08 21:00:00,320.0,0,0,0
1389,2021-07-08 22:00:00,320.0,0,0,0
1390,2021-07-08 23:00:00,320.0,0,0,0


## Playing with already-submitted data

### Add value to whole df

In [20]:
k = pd.read_csv('rf_min-leaf-6_no-zero_320_fixed.csv')

In [16]:
def add_value(df):
    # function to apply
    def f(x):
        if x>320:
            return x+20
        else:
            return x
    df['dangjin_floating'] = df['dangjin_floating'].apply(lambda x:f(x))
    return df

add_value(k).to_csv('rf_min-leaf-6_no-zero_320_fixed_add15.csv',index=False)
# add 20: 8.4 -> 8.01
# add 30: 8.4 -> 8.09

### Add value to certain peaktime

In [21]:
# peaktime add 20
# peaktime: 12~15시
ADD = 20

for i in range(k.shape[0]):
    if i%24 in [11,12,13,14]: # index 0 corresponds to 01:00:00
        k.loc[i,'dangjin_floating'] += ADD

# add tp 20: 8.4 -> 8.27

In [22]:
k.to_csv('rf_min-leaf-6_no-zero_320_fixed_addpt20.csv',index=False)